# Setup/Test Environment:

In [1]:
from web_footprinting_utils import setup_ipython
import os

get_ipython, INTERACTIVE = setup_ipython()

In [2]:
%env TARGET=10.10.220.12

env: TARGET=10.10.220.12


In [3]:
!echo Make sure we are connected to VPN:
result = !ip a | grep tun0

if not len(result)>0:
    print("Not connected!")
    input()
else:
    print("Connected!")
    print(result)

Make sure we are connected to VPN:
Not connected!


# Initial (Unreliable) RustScan:

In [4]:
import os
pwd = os.getcwd()
rustscan = "docker run -v '%s':/workspace  --volume /etc/passwd:/etc/passwd:ro --volume /etc/group:/etc/group:ro --user $(id -u) --rm my-other-github-account/rustscan" % pwd

In [5]:
!{rustscan} -t 4000 -r 1-65535 -a {os.environ["TARGET"]} -- -Pn -A -sV

Unable to find image 'dnola/rustscan:latest' locally
docker: Error response from daemon: pull access denied for dnola/rustscan, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.
See 'docker run --help'.


# Initial Nmap:

In [6]:
!echo $TARGET
!mkdir -p scans/quick scans/tcp scans/udp scans/scripts scans/http scans/no_ping

10.10.220.12


In [7]:
!ping -c 5 $TARGET

PING 10.10.220.12 (10.10.220.12) 56(84) bytes of data.

--- 10.10.220.12 ping statistics ---
5 packets transmitted, 0 received, 100% packet loss, time 4093ms



# Quick Scan:

In [8]:
!nmap -vv -oA scans/quick/quick_scan $TARGET


Starting Nmap 7.60 ( https://nmap.org ) at 2021-09-26 18:24 BST
Initiating ARP Ping Scan at 18:24
Scanning 10.10.220.12 [1 port]
Completed ARP Ping Scan at 18:24, 0.22s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 18:24
Completed Parallel DNS resolution of 1 host. at 18:24, 0.00s elapsed
Initiating SYN Stealth Scan at 18:24
Scanning ip-10-10-220-12.eu-west-1.compute.internal (10.10.220.12) [1000 ports]
Discovered open port 3389/tcp on 10.10.220.12
Discovered open port 21/tcp on 10.10.220.12
Discovered open port 9999/tcp on 10.10.220.12
Completed SYN Stealth Scan at 18:24, 7.56s elapsed (1000 total ports)
Nmap scan report for ip-10-10-220-12.eu-west-1.compute.internal (10.10.220.12)
Host is up, received arp-response (0.00033s latency).
Scanned at 2021-09-26 18:24:17 BST for 8s
Not shown: 997 filtered ports
Reason: 997 no-responses
PORT     STATE SERVICE       REASON
21/tcp   open  ftp           syn-ack ttl 128
3389/tcp open  ms-wbt-server syn-ack ttl 128
999

# All Ports:

In [9]:
!nmap -p- -sV -vv -oA  scans/tcp/tcp_scan $TARGET


Starting Nmap 7.60 ( https://nmap.org ) at 2021-09-26 18:24 BST
NSE: Loaded 42 scripts for scanning.
Initiating ARP Ping Scan at 18:24
Scanning 10.10.220.12 [1 port]
Completed ARP Ping Scan at 18:24, 0.23s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 18:24
Completed Parallel DNS resolution of 1 host. at 18:24, 0.00s elapsed
Initiating SYN Stealth Scan at 18:24
Scanning ip-10-10-220-12.eu-west-1.compute.internal (10.10.220.12) [65535 ports]
Discovered open port 3389/tcp on 10.10.220.12
Discovered open port 21/tcp on 10.10.220.12
Increasing send delay for 10.10.220.12 from 0 to 5 due to 11 out of 30 dropped probes since last increase.
SYN Stealth Scan Timing: About 4.36% done; ETC: 18:36 (0:11:20 remaining)
Increasing send delay for 10.10.220.12 from 5 to 10 due to 11 out of 29 dropped probes since last increase.
SYN Stealth Scan Timing: About 6.31% done; ETC: 18:40 (0:15:06 remaining)
Increasing send delay for 10.10.220.12 from 10 to 20 due to 11 out of 31 d


KeyboardInterrupt



In [10]:
import xmltodict

with open('scans/tcp/tcp_scan.xml','r') as f:
    xml_dict = xmltodict.parse(f.read())

ExpatError: no element found: line 32, column 0

In [11]:
PING=True
try:
    port_list = [port['@portid'] for port in xml_dict['nmaprun']['host']['ports']['port']]
    %env PORTS={",".join(port_list)}
except:
    print("Host is down")
    PING=False

Host is down


# No Ping:

In [12]:
!nmap -sV -Pn -vv -oA scans/no_ping/no_ping_scan $TARGET


Starting Nmap 7.60 ( https://nmap.org ) at 2021-09-26 18:35 BST
NSE: Loaded 42 scripts for scanning.
Initiating ARP Ping Scan at 18:35
Scanning 10.10.220.12 [1 port]
Completed ARP Ping Scan at 18:35, 0.22s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 18:35
Completed Parallel DNS resolution of 1 host. at 18:35, 0.12s elapsed
Initiating SYN Stealth Scan at 18:35
Scanning ip-10-10-220-12.eu-west-1.compute.internal (10.10.220.12) [1000 ports]
Discovered open port 21/tcp on 10.10.220.12
Discovered open port 9999/tcp on 10.10.220.12
Completed SYN Stealth Scan at 18:35, 17.02s elapsed (1000 total ports)
Initiating Service scan at 18:35



KeyboardInterrupt



In [ ]:
if not PING:
    !nmap -sV -Pn -p- -vv -oA  scans/tcp/tcp_scan $TARGET


Starting Nmap 7.60 ( https://nmap.org ) at 2021-09-26 18:36 BST
NSE: Loaded 42 scripts for scanning.
Initiating ARP Ping Scan at 18:36
Scanning 10.10.220.12 [1 port]
Completed ARP Ping Scan at 18:36, 0.22s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 18:36
Completed Parallel DNS resolution of 1 host. at 18:36, 0.00s elapsed
Initiating SYN Stealth Scan at 18:36
Scanning ip-10-10-220-12.eu-west-1.compute.internal (10.10.220.12) [65535 ports]
Discovered open port 21/tcp on 10.10.220.12
Increasing send delay for 10.10.220.12 from 0 to 5 due to 11 out of 32 dropped probes since last increase.
SYN Stealth Scan Timing: About 2.93% done; ETC: 18:53 (0:17:08 remaining)
Increasing send delay for 10.10.220.12 from 5 to 10 due to 11 out of 27 dropped probes since last increase.
SYN Stealth Scan Timing: About 4.50% done; ETC: 18:58 (0:21:36 remaining)
SYN Stealth Scan Timing: About 5.65% done; ETC: 19:02 (0:25:21 remaining)
SYN Stealth Scan Timing: About 6.85% done; ETC

# Run Default Scripts:

In [ ]:
if PING:
    !nmap -sC -p $PORTS -vv -oA scans/scripts/script_scan $TARGET

In [ ]:
if PING:
    !nmap -A -T5 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T5 -Pn -vv --script=default,safe,discovery $TARGET

In [ ]:
if PING:
    !nmap -A -T3 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T3 -Pn -vv --script=default,safe,discovery $TARGET

In [ ]:
from getpass import getpass
import os

if PING:
    !echo { getpass() } | sudo -S nmap -A -p- -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}
else:
    !echo { getpass() } | sudo -S nmap -A -p- -Pn -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}

In [ ]:
# UDP Scan, this takes forever...

# from getpass import getpass

# !echo { getpass() } > tmp.txt
# !cat tmp.txt | sudo -S nmap -sU --open -oA scans/udp/udp_scan $TARGET
# !cat tmp.txt | sudo -S nmap -sU --open -p- -oA scans/udp/udp_scan $TARGET
# !rm -rf tmp.txt